In [0]:
import numpy as np
from sklearn import metrics
import pandas as pd
import matplotlib.pyplot as plt
import scipy.spatial
from sklearn.utils import shuffle
from nltk.metrics import precision, recall, f_measure
import scipy.stats as st

In [2]:
!pip install sentence_transformers

     |████████████████████████████████| 61kB 2.1MB/s 
     |████████████████████████████████| 675kB 8.4MB/s 
     |████████████████████████████████| 3.8MB 16.7MB/s 
     |████████████████████████████████| 890kB 33.7MB/s 
     |████████████████████████████████| 1.1MB 46.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=7dbbeae3effb11394e684a3d27f9898f054027e9a3708a6d73539457b86d8cd7
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b94f5a9afee2ec9eb19c76dbba78d0cb8c728d7d2a3315006c227b0e9296e479
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [0]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
data = pd.read_csv('/content/drive/My Drive/data_emb/dataset.csv',header = None)

In [0]:
vectorizer = CountVectorizer()

In [0]:
my_vectors = data[[0,1]]

In [0]:
row1 = data[0]
row2=data[1]

In [0]:
len(row1)

2638

In [0]:
row1 = row1[:500]
row2 = row2[:500]

In [0]:
len(row1)

500

In [0]:
corpus_embeddings = vectorizer.fit_transform(row1)
query_embeddings =vectorizer.fit_transform(row2)

In [0]:
X1 = corpus_embeddings.toarray()
X2 = query_embeddings.toarray()

In [0]:
data_test = data[2]

In [0]:
data_test = data_test[:500]

In [0]:
def get_fscore(data, threshold=0.8, method = "CountVectorizer"):
  row1 = data[0]
  row2 = data[1]
  results = []
  num_sentence = 0
  len_rows = len(row1)

  if method == "use":
      embedder = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
  elif method == "sroberta":
      embedder = SentenceTransformer('roberta-base-nli-mean-tokens')
  elif method == "sbert":
      embedder = SentenceTransformer('bert-base-nli-mean-tokens')

  elif method == "CountVectorizer":
      vectorizer = CountVectorizer()

  for i in range(0,len_rows):
    if method in ("sbert", "sroberta"):
        row1emb = embedder.encode([row1[i], row2[i]] )
    elif method == "use":
        row1emb = embedder([row1[i], row2[i]] )
    else:
        row1emb = vectorizer.fit_transform([row1[i], row2[i]] )

    distances = metrics.pairwise.cosine_similarity(row1emb[0:1], row1emb[1:2])[0, 0]
    if (distances > threshold):
          results.append(1)
    else:
          results.append(0)
  return results

In [0]:
data_test = data_test[:500]

для порога = 0.8

In [0]:
f1measure = metrics.f1_score(data_test, get_fscore(row1, row2, threshold=0.8))
print(f1measure)

0.8468468468468467


для порога = 0.9


In [0]:
f1measure = metrics.f1_score(data_test, get_fscore(row1, row2, threshold=0.9))
print(f1measure)

0.6940451745379876


### CountVect

In [0]:
data = pd.read_csv('/content/drive/My Drive/data_emb/dataset.csv',header = None)

In [0]:
f1measure = metrics.f1_score(data[2], get_fscore(data, threshold=0.8))
print(f1measure)

0.9008130081300814


In [0]:
f1_arr = []
for i in range(100, 2600, 100):
  separ = i
  data = shuffle(data)
  data.reset_index(inplace=True, drop=True)
  f1measure = metrics.f1_score(data[2][:separ], get_fscore(data[:separ], threshold=0.8))
  f1_arr.append(f1measure)

In [43]:
import numpy
mean_f1 = numpy.mean(f1_arr)
std_f1 = numpy.std(f1_arr)

print('Mean:', mean_f1)
print('Standard Deviation:', std_f1)

Mean: 0.9005196531057171
Standard Deviation: 0.007343019403532


In [0]:
import numpy
mean_f1 = numpy.mean(f1_arr)
std_f1 = numpy.std(f1_arr)

print('Mean:', mean_f1)
print('Standard Deviation:', std_f1)

Mean: 0.8842186358800688
Standard Deviation: 0.03040012025566086


In [0]:
interval = st.norm.interval(
            0.95, loc=np.mean(f1_arr), scale=np.std(f1_arr))

print((interval[1]-interval[0])/2)

0.059583140826781844


### roberta

In [0]:
data = pd.read_csv('/content/drive/My Drive/data_emb/dataset.csv',header = None)

In [0]:
f1measure = metrics.f1_score(data[2], get_fscore(data, threshold=0.8, method = "sroberta"))
print(f1measure)

100%|██████████| 459M/459M [00:24<00:00, 19.0MB/s]


0.9263466471234886


In [0]:
f1_arr = []
for i in range(100, 2600, 100):
  separ = i
  data = shuffle(data)
  data.reset_index(inplace=True, drop=True)
  f1measure = metrics.f1_score(data[2][:separ], get_fscore(data[:separ], threshold=0.8, method = "sroberta"))
  f1_arr.append(f1measure)

In [0]:
mean_f1 = numpy.mean(f1_arr)
std_f1 = numpy.std(f1_arr)

print('Mean:', mean_f1)
print('Standard Deviation:', std_f1)

Mean: 0.8813148563612752
Standard Deviation: 0.03281033117526479


In [0]:
interval = st.norm.interval(
            0.95, loc=np.mean(f1_arr), scale=np.std(f1_arr))

print((interval[1]-interval[0])/2)

0.06430706742435077


### USE

In [0]:
f1measure = metrics.f1_score(data[2], get_fscore(data, threshold=0.8, method = "use"))
print(f1measure)

0.8708733806936899


In [0]:
f1_arr = []
for i in range(100, 2600, 100):
  separ = i
  data = shuffle(data)
  data.reset_index(inplace=True, drop=True)
  f1measure = metrics.f1_score(data[2][:separ], get_fscore(data[:separ], threshold=0.8, method = "use"))
  f1_arr.append(f1measure)

In [0]:
mean_f1 = numpy.mean(f1_arr)
std_f1 = numpy.std(f1_arr)

print('Mean:', mean_f1)
print('Standard Deviation:', std_f1)

Mean: 0.8751586468604882
Standard Deviation: 0.03250012225566086


In [0]:
st.norm.interval(
            0.95, loc=np.mean(f1_arr), scale=np.std(f1_arr))

(0.8114595777462442, 0.9388577159747321)

In [0]:
f1_arr

[1.0,
 0.8834355828220859,
 0.8780487804878049,
 0.8866498740554157,
 0.8854166666666666,
 0.8305555555555555,
 0.8370535714285714,
 0.8411214953271027,
 0.8411214953271027,
 0.8402489626556017,
 0.8402489626556017,
 0.8508384819064431,
 0.8715251690458301,
 0.8875816993464052,
 0.8961114335461405,
 0.8959276018099548,
 0.8918918918918919,
 0.8878923766816144,
 0.8864017907106884,
 0.8830624016780283,
 0.8765670202507233,
 0.8735529830810329,
 0.87123745819398,
 0.87123745819398,
 0.87123745819398]

### bert

In [0]:
f1measure = metrics.f1_score(data[2], get_fscore(data, threshold=0.8, method = "sbert"))
print(f1measure) 

0.9092203631185476


In [0]:
f1_arr = []
for i in range(100, 2600, 100):
  separ = i
  data = shuffle(data)
  data.reset_index(inplace=True, drop=True)
  f1measure = metrics.f1_score(data[2][:separ], get_fscore(data[:separ], threshold=0.8, method = "sbert"))
  f1_arr.append(f1measure)

In [0]:
mean_f1 = numpy.mean(f1_arr)
std_f1 = numpy.std(f1_arr)

print('Mean:', mean_f1)
print('Standard Deviation:', std_f1)

Mean: 0.8614615538839357
Standard Deviation: 0.03290312284509809


In [0]:
interval = st.norm.interval(
            0.95, loc=np.mean(f1_arr), scale=np.std(f1_arr))

print((interval[1]-interval[0])/2)

0.06448893575528936
